## Capstone Project - The Battle of Neighborhoods Week2

##  Data section

After importing the necessary libraries, we download the data from the HM Land Registry website as follows:

In [53]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

Before using data, we will have to explore and understand it.

#### 2. Explore and Understand Data

We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows: 

In [3]:
df_ppd.head(5) 

,{726BF13A-993E-0A46-E053-6C04A8C01D0D},75000,2018-06-20 00:00,DL13 4BA,T,N,F,21,Unnamed: 8,HIGHFIELDS,TOW LAW,BISHOP AUCKLAND,COUNTY DURHAM,COUNTY DURHAM.1,A,A.1
0,{726BF13A-993F-0A46-E053-6C04A8C01D0D},115000,2018-07-13 00:00,DL17 9LB,S,N,F,4,NaN,THE LANE,WEST CORNFORTH,FERRYHILL,COUNTY DURHAM,COUNTY DURHAM,A,A
1,{726BF13A-9940-0A46-E053-6C04A8C01D0D},24000,2018-04-10 00:00,SR7 9AG,F,N,L,20B,NaN,WOODS TERRACE,MURTON,SEAHAM,COUNTY DURHAM,COUNTY DURHAM,A,A
2,{726BF13A-9941-0A46-E053-6C04A8C01D0D},56000,2018-06-22 00:00,DL5 5PS,T,N,F,6,NaN,HEILD CLOSE,NaN,NEWTON AYCLIFFE,COUNTY DURHAM,COUNTY DURHAM,A,A
3,{726BF13A-9942-0A46-E053-6C04A8C01D0D},220000,2018-05-25 00:00,DL16 7HE,D,N,F,25,NaN,BECKWITH CLOSE,KIRK MERRINGTON,SPENNYMOOR,COUNTY DURHAM,COUNTY DURHAM,A,A
4,{726BF13A-9943-0A46-E053-6C04A8C01D0D},58000,2018-05-09 00:00,DL14 6FH,F,N,L,23,NaN,AINTREE DRIVE,NaN,BISHOP AUCKLAND,COUNTY DURHAM,COUNTY DURHAM,A,A


In [4]:
df_ppd.shape

(993670, 16)

Our dataset consists of over 700000 rows and 16 columns. We will now prepare and preprocess data accordingly. 

#### 3. Data preparation and preprocessing

At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps: 
- Rename the column names 
- Format the date column 
- Sort data by date of sale 
- Select data only for the city of London 
- Make a list of street names in London 
- Calculate the street-wise average price of the property
- Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names 
- Join the data to find the coordinates of locations which fit into client's budget
- Plot recommended locations on London map along with current market prices






In [5]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [6]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [7]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [8]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [9]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [10]:
# Display the dataframe
df_affordable

,Street,Avg_Price
194,ALBION SQUARE,2.450000e+06
388,ANHALT ROAD,2.435000e+06
402,ANSDELL TERRACE,2.250000e+06
417,APPLEGARTH ROAD,2.400000e+06
845,BARONSMEAD ROAD,2.375000e+06
969,BEAUCLERC ROAD,2.480000e+06
1091,BELVEDERE DRIVE,2.340000e+06
1204,BICKENHALL STREET,2.208500e+06
1242,BIRCHLANDS AVENUE,2.217000e+06
1539,BRAMPTON GROVE,2.456875e+06


In [11]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [12]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 194
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 194, dtype: object
item.Street only: ALBION SQUARE
index: 388
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 388, dtype: object
item.Street only: ANHALT ROAD
index: 402
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 402, dtype: object
item.Street only: ANSDELL TERRACE
index: 417
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 417, dtype: object
item.Street only: APPLEGARTH ROAD
index: 845
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 845, dtype: object
item.Street only: BARONSMEAD ROAD
index: 969
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 969, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1091
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1091, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1204
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [13]:
geolocator = Nominatim()


C:\Users\JOSIA MOTSEO\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [14]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

C:\Users\JOSIA MOTSEO\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
df_affordable

,Street,Avg_Price,city_coord
194,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)"
388,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)"
402,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
417,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)"
845,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
969,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)"
1091,BELVEDERE DRIVE,2.340000e+06,"(38.201316, -84.623076)"
1204,BICKENHALL STREET,2.208500e+06,"(51.5211969, -0.1589341)"
1242,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)"
1539,BRAMPTON GROVE,2.456875e+06,"(51.5703648, -0.2833944)"


In [16]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

C:\Users\JOSIA MOTSEO\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [17]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
194,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)",-41.273758,173.289393
388,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)",51.480326,-0.166761
402,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
417,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)",53.748654,-0.326670
845,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
969,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)",51.499577,-0.229033
1091,BELVEDERE DRIVE,2.340000e+06,"(38.201316, -84.623076)",38.201316,-84.623076
1204,BICKENHALL STREET,2.208500e+06,"(51.5211969, -0.1589341)",51.521197,-0.158934
1242,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468
1539,BRAMPTON GROVE,2.456875e+06,"(51.5703648, -0.2833944)",51.570365,-0.283394


In [18]:
df = df_affordable.drop(columns=['city_coord'])


In [19]:
df

,Street,Avg_Price,Latitude,Longitude
194,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
388,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
402,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
417,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
845,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
969,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033
1091,BELVEDERE DRIVE,2.340000e+06,38.201316,-84.623076
1204,BICKENHALL STREET,2.208500e+06,51.521197,-0.158934
1242,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468
1539,BRAMPTON GROVE,2.456875e+06,51.570365,-0.283394


In [20]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

C:\Users\JOSIA MOTSEO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [21]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [22]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'K2KHBXRAX3YPPPOCELWV5JWPFGF4IAV1JAFJYYJNVQJ1WONH' # Foursquare ID
CLIENT_SECRET = 'BGOM4RAMKJ5Q0KQ1AQ1EKOUHYDR5QWBO1ETDTDKPQVUNBBK0' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K2KHBXRAX3YPPPOCELWV5JWPFGF4IAV1JAFJYYJNVQJ1WONH
CLIENT_SECRET:BGOM4RAMKJ5Q0KQ1AQ1EKOUHYDR5QWBO1ETDTDKPQVUNBBK0


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

#### 4. Modeling

After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALEDONIAN ROAD
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL ROAD
HOXTON SQUARE
HUNTER ROAD
JACKSON

In [25]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
4,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
5,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
6,ALBION SQUARE,-41.273758,173.289393,Burger Culture,-41.274750,173.284030,Burger Joint
7,ALBION SQUARE,-41.273758,173.289393,Mango,-41.274460,173.285345,Indian Restaurant
8,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
9,ALBION SQUARE,-41.273758,173.289393,Fresh Choice,-41.272194,173.287218,Supermarket


In [26]:
location_venues.groupby('Street').count()


,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,29,29,29,29,29,29
ANHALT ROAD,14,14,14,14,14,14
ANSDELL TERRACE,56,56,56,56,56,56
APPLEGARTH ROAD,5,5,5,5,5,5
BARONSMEAD ROAD,15,15,15,15,15,15
BEAUCLERC ROAD,29,29,29,29,29,29
BICKENHALL STREET,99,99,99,99,99,99
BIRCHLANDS AVENUE,11,11,11,11,11,11
BRAMPTON GROVE,4,4,4,4,4,4


In [27]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))


There are 336 uniques categories.


In [28]:
location_venues.shape

(5465, 7)

In [29]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.034483,0.000000,...,0.0000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
1,ANHALT ROAD,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,ANSDELL TERRACE,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.017857,0.000000,...,0.0000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.017857,0.000000,0.00
3,APPLEGARTH ROAD,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
4,BARONSMEAD ROAD,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
5,BEAUCLERC ROAD,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
6,BICKENHALL STREET,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.010101,0.000000,...,0.0000,0.010101,0.0,0.000000,0.00,0.010101,0.000000,0.010101,0.010101,0.00
7,BIRCHLANDS AVENUE,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
8,BRAMPTON GROVE,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
9,BRIARDALE GARDENS,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00


In [31]:
london_grouped.shape


(142, 337)

In [32]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.21
1         Restaurant  0.07
2        Coffee Shop  0.07
3  Indian Restaurant  0.07
4                Bar  0.07


----ANHALT ROAD----
                 venue  freq
0                  Pub  0.29
1        Grocery Store  0.14
2                Diner  0.07
3         Cocktail Bar  0.07
4  Japanese Restaurant  0.07


----ANSDELL TERRACE----
                venue  freq
0  Italian Restaurant  0.07
1      Clothing Store  0.07
2                 Pub  0.05
3   Indian Restaurant  0.05
4               Hotel  0.05


----APPLEGARTH ROAD----
       venue  freq
0        Pub   0.4
1     Casino   0.2
2  Nightclub   0.2
3        Bar   0.2
4     Palace   0.0


----BARONSMEAD ROAD----
               venue  freq
0  Food & Drink Shop  0.13
1         Restaurant  0.13
2    Nature Preserve  0.07
3        Coffee Shop  0.07
4      Movie Theater  0.07


----BEAUCLERC ROAD----
             venue  freq
0            Hotel  0.17
1              Pub  0.1

4              Garden  0.05


----GERARD ROAD----
           venue  freq
0            Pub  0.12
1     Restaurant  0.08
2           Café  0.08
3    Pizza Place  0.08
4  Grocery Store  0.08


----GERRARD ROAD----
                  venue  freq
0                   Pub  0.08
1           Coffee Shop  0.07
2  Gym / Fitness Center  0.04
3     French Restaurant  0.04
4          Burger Joint  0.04


----GIRDLERS ROAD----
                      venue  freq
0                       Pub  0.12
1        Italian Restaurant  0.08
2  Bike Rental / Bike Share  0.05
3         Convention Center  0.05
4                      Park  0.05


----GORDON PLACE----
             venue  freq
0      Pizza Place  0.18
1         Bus Stop  0.09
2    Deli / Bodega  0.09
3      Bus Station  0.09
4  Thai Restaurant  0.09


----GRAFTON SQUARE----
          venue  freq
0           Pub  0.09
1  Burger Joint  0.06
2   Coffee Shop  0.05
3  Cocktail Bar  0.05
4    Restaurant  0.05


----GRAHAM TERRACE----
                venue  fre

                 venue  freq
0                 Café  0.12
1             Boutique  0.11
2   Italian Restaurant  0.08
3                Hotel  0.07
4  Japanese Restaurant  0.04


----PEMBRIDGE MEWS----
                venue  freq
0                 Pub  0.08
1  Italian Restaurant  0.06
2              Bakery  0.04
3          Restaurant  0.03
4       Grocery Store  0.03


----PEMBRIDGE ROAD----
                   venue  freq
0           Soccer Field   1.0
1      Accessories Store   0.0
2                 Palace   0.0
3           Perfume Shop   0.0
4  Performing Arts Venue   0.0


----PEMBROKE STUDIOS----
         venue  freq
0   Restaurant  0.12
1         Café  0.06
2   Sports Bar  0.06
3          Pub  0.06
4  Supermarket  0.06


----PENCOMBE MEWS----
                venue  freq
0  Italian Restaurant  0.09
1                 Pub  0.07
2         Coffee Shop  0.05
3      Clothing Store  0.04
4          Restaurant  0.03


----PETERSHAM PLACE----
                     venue  freq
0                 

              venue  freq
0               Pub  0.22
1              Café  0.13
2       Coffee Shop  0.09
3  Asian Restaurant  0.04
4             Trail  0.04


----WILSON STREET----
              venue  freq
0  Business Service  0.25
1      Liquor Store  0.25
2           Gay Bar  0.25
3     Train Station  0.25
4      Perfume Shop  0.00


----WINCHENDON ROAD----
                venue  freq
0                Café  0.12
1  Italian Restaurant  0.09
2         Coffee Shop  0.09
3                 Pub  0.07
4              Bakery  0.07


----WINGATE ROAD----
                           venue  freq
0                Harbor / Marina   1.0
1              Accessories Store   0.0
2  Paper / Office Supplies Store   0.0
3             Persian Restaurant   0.0
4                   Perfume Shop   0.0




In [33]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))



In [35]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)



In [36]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Pub,Restaurant,Coffee Shop,Indian Restaurant,Bar,Bookstore,Supermarket,Brewery,Burger Joint
1,ANHALT ROAD,Pub,Grocery Store,Japanese Restaurant,Gym / Fitness Center,Plaza,Pizza Place,English Restaurant,Cocktail Bar,Garden,Diner
2,ANSDELL TERRACE,Clothing Store,Italian Restaurant,Pub,Indian Restaurant,Hotel,French Restaurant,English Restaurant,Bakery,Juice Bar,Restaurant
3,APPLEGARTH ROAD,Pub,Casino,Nightclub,Bar,Zoo,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop
4,BARONSMEAD ROAD,Food & Drink Shop,Restaurant,Coffee Shop,Pizza Place,Nature Preserve,Movie Theater,Breakfast Spot,Park,Thai Restaurant,Community Center


In [37]:
venues_sorted.shape

(142, 11)

In [38]:
london_grouped.shape

(142, 337)

In [39]:
london_grouped=df

After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby. 

In [40]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 4, 0, 4, 2, 1, 2, 0, 0, 1, 4, 4, 4, 1, 4, 2, 2, 1, 4, 0, 1, 3,
       3, 4, 1, 1, 0, 4, 3, 0, 4, 2, 2, 2, 3, 4, 4, 2, 0, 2, 2, 3, 0, 4,
       0, 3, 0, 0, 4, 2])

In [41]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
194,ALBION SQUARE,2450000.0,-41.273758,173.289393
388,ANHALT ROAD,2435000.0,51.480326,-0.166761
402,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
417,APPLEGARTH ROAD,2400000.0,53.748654,-0.326670
845,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [42]:
london_grouped_clustering.shape

(152, 4)

In [43]:
df.shape

(152, 4)

In [44]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [45]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [46]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
194,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Pub,Restaurant,Coffee Shop,Indian Restaurant,Bar,Bookstore,Supermarket,Brewery,Burger Joint
388,ANHALT ROAD,2.435000e+06,51.480326,-0.166761,4,Pub,Grocery Store,Japanese Restaurant,Gym / Fitness Center,Plaza,Pizza Place,English Restaurant,Cocktail Bar,Garden,Diner
402,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,0,Clothing Store,Italian Restaurant,Pub,Indian Restaurant,Hotel,French Restaurant,English Restaurant,Bakery,Juice Bar,Restaurant
417,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,4,Pub,Casino,Nightclub,Bar,Zoo,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop
845,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,2,Food & Drink Shop,Restaurant,Coffee Shop,Pizza Place,Nature Preserve,Movie Theater,Breakfast Spot,Park,Thai Restaurant,Community Center
969,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033,1,Hotel,Pub,Coffee Shop,Thai Restaurant,Chinese Restaurant,Grocery Store,Italian Restaurant,Falafel Restaurant,Cocktail Bar,Fish & Chips Shop
1091,BELVEDERE DRIVE,2.340000e+06,38.201316,-84.623076,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1204,BICKENHALL STREET,2.208500e+06,51.521197,-0.158934,0,Hotel,Restaurant,Café,Chinese Restaurant,Pub,Italian Restaurant,Gastropub,Indian Restaurant,Coffee Shop,Movie Theater
1242,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,0,Pub,French Restaurant,Coffee Shop,Chinese Restaurant,Bakery,Brewery,Lake,Breakfast Spot,Train Station,Food Court
1539,BRAMPTON GROVE,2.456875e+06,51.570365,-0.283394,1,Men's Store,Bar,Middle Eastern Restaurant,Lake,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop


In [47]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [48]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
402,2250000.0,Clothing Store,Italian Restaurant,Pub,Indian Restaurant,Hotel,French Restaurant,English Restaurant,Bakery,Juice Bar,Restaurant
1204,2208500.0,Hotel,Restaurant,Café,Chinese Restaurant,Pub,Italian Restaurant,Gastropub,Indian Restaurant,Coffee Shop,Movie Theater
1242,2217000.0,Pub,French Restaurant,Coffee Shop,Chinese Restaurant,Bakery,Brewery,Lake,Breakfast Spot,Train Station,Food Court
2203,2200000.0,Nightlife Spot,Soccer Field,Playground,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
2612,2250000.0,Gastropub,Health & Beauty Service,Construction & Landscaping,Food Court,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop


In [49]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
194,2450000.0,Café,Pub,Restaurant,Coffee Shop,Indian Restaurant,Bar,Bookstore,Supermarket,Brewery,Burger Joint
969,2480000.0,Hotel,Pub,Coffee Shop,Thai Restaurant,Chinese Restaurant,Grocery Store,Italian Restaurant,Falafel Restaurant,Cocktail Bar,Fish & Chips Shop
1539,2456875.0,Men's Store,Bar,Middle Eastern Restaurant,Lake,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
1960,2492500.0,Supermarket,English Restaurant,Pub,Park,Gym,Rental Car Location,Hardware Store,Café,American Restaurant,Food
2114,2461000.0,Bus Station,Soccer Field,Spa,Windmill,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food


In [50]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
845,2375000.0,Food & Drink Shop,Restaurant,Coffee Shop,Pizza Place,Nature Preserve,Movie Theater,Breakfast Spot,Park,Thai Restaurant,Community Center
1091,2340000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2046,2375000.0,Pub,Bakery,Ice Cream Shop,Grocery Store,Indian Restaurant,Park,Bookstore,Yoga Studio,Hotel,Italian Restaurant
2107,2379652.7,Pub,Bakery,Ice Cream Shop,Grocery Store,Yoga Studio,Hostel,Coffee Shop,Hotel,Indian Restaurant,Bookstore
2914,2367500.0,Hotel,Pub,Garden,Italian Restaurant,Coffee Shop,Café,Indian Restaurant,Restaurant,Bar,Supermarket


In [51]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2220,2.300000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2382,2.286679e+06,Café,Pub,Bar,Pizza Place,Italian Restaurant,Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Furniture / Home Store,Supermarket
2660,2.287500e+06,Pub,Gift Shop,Brewery,Art Museum,Gym / Fitness Center,Zoo,Food Court,Fish & Chips Shop,Fish Market,Flea Market
3344,2.298000e+06,Hotel,Zoo,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop
4241,2.265000e+06,Pub,Zoo,Food Truck,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop


In [52]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
388,2435000.0,Pub,Grocery Store,Japanese Restaurant,Gym / Fitness Center,Plaza,Pizza Place,English Restaurant,Cocktail Bar,Garden,Diner
417,2400000.0,Pub,Casino,Nightclub,Bar,Zoo,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop
1617,2397132.0,Sporting Goods Shop,Indian Restaurant,Coffee Shop,Grocery Store,Health & Beauty Service,Fried Chicken Joint,French Restaurant,Fast Food Restaurant,Filipino Restaurant,Furniture / Home Store
1781,2400000.0,Park,Zoo,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop
1896,2445000.0,Café,Athletics & Sports,Garden Center,Gas Station,Gastropub,Stadium,Brewery,Grocery Store,Fish & Chips Shop,Bakery


## Results and Discussion section

First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.

We may discuss our results under two main perspectives. 

First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer. 


## Conclusion

To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario? 

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments. 

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer. 

